In [1]:
import os
import json
import requests
import pandas as pd
import pytz
from io import BytesIO
from datetime import datetime, timedelta
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import googleapiclient.http


In [2]:
# ===========================
# CONFIGURATION & AUTHENTICATION
# ===========================

# Configuration
SERVICE_ACCOUNT_FILE = "/Users/sachin/TheJuniorDataScientist/credentials/sachin_service account.json"
SCOPES = ['https://www.googleapis.com/auth/drive']
CHECKPOINT_FILE = "download_progress.json"
CHUNKSIZE = 10 * 1024 * 1024  # 10 MB
SAVE_INTERVAL = 50 * 1024 * 1024  # 50 MB


# Authenticate and build the Drive API client
def authenticate_drive_service():
    try:
        credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        drive_service = build('drive', 'v3', credentials=credentials)
        print("✅ Google Drive service authorized successfully.")
        return drive_service
    except Exception as e:
        print("❌ Error during authentication:", e)
        raise


In [3]:
# ===========================
# DATE RANGE HANDLING
# ===========================

# Generate date range (YYYYMMDD format)
def get_date_range(local_tz, start_date=None, end_date=None, days=None):
    current_time = datetime.now(pytz.utc).astimezone(local_tz)
    if end_date:
        end_date = datetime.strptime(end_date, "%Y%m%d").astimezone(local_tz)
    else:
        end_date = current_time

    if start_date:
        start_date = datetime.strptime(start_date, "%Y%m%d").astimezone(local_tz)
    elif days:
        start_date = end_date - timedelta(days=days)
    else:
        start_date = end_date - timedelta(days=10)

    return start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d")


In [4]:
# ===========================
# FETCH FILES FROM GOOGLE DRIVE
# ===========================

def fetch_files_with_dates(drive_service, folder_id, date_range):
    try:
        query = f"'{folder_id}' in parents and mimeType='text/csv' and trashed=false"
        files = []
        page_token = None

        while True:
            response = drive_service.files().list(
                q = query,
                fields = "nextPageToken, files(id, name, size)",
                pageToken = page_token
            ).execute()
            
            files.extend(response.get('files', []))
            page_token = response.get('nextPageToken')
            if not page_token:
                break  

        print(f"📂 Total files found in the folder: {len(files)}")

        # Filter files by date
        filtered_files = [
            {**file, 'size_mb': round(int(file.get('size', 0)) / (1024 * 1024), 2)} for file in files if any(date in file['name'] for date in date_range)
        ]

        print(f"📅 Number of files matching the date range: {len(filtered_files)}")
        return filtered_files
    except HttpError as error:
        print(f"❌ An error occurred: {error}")
        raise


In [5]:
from tqdm import tqdm

def process_files(drive_service, files, required_columns):
    combined_data = []
    summary_report = {"Resumed Files": [], "Fully Downloaded Files": [], "Skipped Files": []}

    try:
        with open(CHECKPOINT_FILE, "r") as f:
            download_progress = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        download_progress = {}

    for file in files:
        file_id = file['id']
        file_name = file['name']
        file_size = int(file.get('size', 0))
        bytes_downloaded = download_progress.get(file_name, 0)

        if bytes_downloaded >= file_size:
            print(f"⏩ {file_name} already downloaded. Skipping...")
            summary_report["Skipped Files"].append(file_name)
            continue

        print(f"\n⬇️ Downloading file: {file_name} ({file_size / (1024 * 1024):.2f} MB)")

        request = drive_service.files().get_media(fileId=file_id)
        file_stream = BytesIO()
        downloader = googleapiclient.http.MediaIoBaseDownload(file_stream, request, chunksize=CHUNKSIZE)

        # **Initialize tqdm progress bar**
        with tqdm(total=file_size, unit='B', unit_scale=True, desc=file_name, position=0, leave=True) as pbar:
            try:
                while True:
                    status, done = downloader.next_chunk()
                    new_bytes = len(file_stream.getvalue()) - bytes_downloaded
                    bytes_downloaded += new_bytes
                    download_progress[file_name] = bytes_downloaded

                    # **Update the progress bar instead of printing logs**
                    pbar.update(new_bytes)

                    # Save progress
                    with open(CHECKPOINT_FILE, "w") as f:
                        json.dump(download_progress, f)

                    if done:
                        file_stream.seek(0)
                        df = pd.read_csv(file_stream, usecols=required_columns)
                        df['source_file'] = file_name
                        combined_data.append(df)
                        summary_report["Fully Downloaded Files"].append(file_name)
                        del download_progress[file_name]
                        with open(CHECKPOINT_FILE, "w") as f:
                            json.dump(download_progress, f)
                        break
            except (requests.ConnectionError, HttpError) as e:
                print(f"❌ Network error during download: {e}. Saving progress...")
                summary_report["Resumed Files"].append(file_name)

    if combined_data:
        final_df = pd.concat(combined_data, ignore_index=True)
        final_df[['to_qty']] = final_df[['to_qty']].fillna(0)
        return final_df, summary_report
    else:
        return pd.DataFrame(), summary_report


In [9]:
! pip3 install tqdm


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3.13 -m pip install --upgrade pip


In [ ]:
# ===========================
# FETCH FILES FROM GOOGLE DRIVE (Supports Multiple File Types)
# ===========================

def fetch_files_with_dates(drive_service, folder_id, date_range, file_types=None):
    """
    Fetches files from Google Drive within the specified folder, date range, and file types.

    Args:
        drive_service (Resource): Google Drive service resource.
        folder_id (str)         : The ID of the Google Drive folder to search.
        date_range (list)       : List of date strings (YYYYMMDD) to filter files by their name.
        file_types (list)       : List of file extensions to filter (e.g., ['csv', 'xlsx']).

    Returns:
        list: A list of files that match the criteria.
    """
    # Supported MIME types for different file extensions
    mime_type_mapping = {
        'csv': 'text/csv',
        'xlsx': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        'xls': 'application/vnd.ms-excel',
        'json': 'application/json',
        'txt': 'text/plain',
        'pdf': 'application/pdf',
        'docx': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
        'doc': 'application/msword',
        'pptx': 'application/vnd.openxmlformats-officedocument.presentationml.presentation',
        'ppt': 'application/vnd.ms-powerpoint',
        'html': 'text/html',
        'zip': 'application/zip',
        'rar': 'application/vnd.rar',
        'tar': 'application/x-tar',
        'gz': 'application/gzip',
        'jpg': 'image/jpeg',
        'jpeg': 'image/jpeg',
        'png': 'image/png',
        'gif': 'image/gif',
        'mp4': 'video/mp4',
        'mp3': 'audio/mpeg' }
    
    # Create a query string for file types if specified
    mime_queries = []
    if file_types:
        for file_type in file_types:
            if file_type in mime_type_mapping:
                mime_queries.append(f"mimeType='{mime_type_mapping[file_type]}'")
            else:
                print(f"⚠️ Warning: Unsupported file type '{file_type}'. Skipping it.")
    
    # Combine file type queries using OR if specified
    file_type_query = f" and ({' or '.join(mime_queries)})" if mime_queries else ""
    
    try:
        query = f"'{folder_id}' in parents and trashed=false{file_type_query}"
        files = []
        page_token = None

        while True:
            response = drive_service.files().list(
                q=query,
                fields="nextPageToken, files(id, name, size)",
                pageToken=page_token
            ).execute()
            
            files.extend(response.get('files', []))
            page_token = response.get('nextPageToken')
            if not page_token:
                break  

        print(f"📂 Total files found in the folder: {len(files)}")

        # Filter files by date in the filename
        filtered_files = [
            {**file, 'size_mb': round(int(file.get('size', 0)) / (1024 * 1024), 2)}
            for file in files
            if any(date in file['name'] for date in date_range)
        ]

        print(f"📅 Number of files matching the date range: {len(filtered_files)}")
        return filtered_files
    except HttpError as error:
        print(f"❌ An error occurred: {error}")
        raise

# ===========================
# FILE PROCESSING & DOWNLOADING
# ===========================

def process_files(drive_service, files, required_columns):
    combined_data = []
    summary_report = {"Resumed Files": [], "Fully Downloaded Files": [], "Skipped Files": []}

    try:
        with open(CHECKPOINT_FILE, "r") as f:
            download_progress = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        download_progress = {}

    for file in files:
        file_id = file['id']
        file_name = file['name']
        file_size = int(file.get('size', 0))
        bytes_downloaded = download_progress.get(file_name, 0)

        if bytes_downloaded >= file_size:
            print(f"⏩ {file_name} already downloaded. Skipping...")
            summary_report["Skipped Files"].append(file_name)
            continue

        print(f"\n⬇️ Downloading file: {file_name} (Resuming from {bytes_downloaded} bytes)")

        request = drive_service.files().get_media(fileId=file_id)
        file_stream = BytesIO()
        downloader = googleapiclient.http.MediaIoBaseDownload(file_stream, request, chunksize=CHUNKSIZE)
        
        try:
            while True:
                status, done = downloader.next_chunk()
                bytes_downloaded += len(file_stream.getvalue())
                download_progress[file_name] = bytes_downloaded

                # Save progress
                with open(CHECKPOINT_FILE, "w") as f:
                    json.dump(download_progress, f)

                print(f"✅ Download {int(status.progress() * 100)}% complete for {file_name}.")
                
                if done:
                    file_stream.seek(0)
                    df = pd.read_csv(file_stream, usecols=required_columns)
                    df['source_file'] = file_name
                    combined_data.append(df)
                    summary_report["Fully Downloaded Files"].append(file_name)
                    del download_progress[file_name]
                    with open(CHECKPOINT_FILE, "w") as f:
                        json.dump(download_progress, f)
                    break
        except (requests.ConnectionError, HttpError) as e:
            print(f"❌ Network error during download: {e}. Saving progress...")
            summary_report["Resumed Files"].append(file_name)

    if combined_data:
        final_df = pd.concat(combined_data, ignore_index=True)
        final_df[['to_qty']] = final_df[['to_qty']].fillna(0)
        return final_df, summary_report
    else:
        return pd.DataFrame(), summary_report


In [6]:
# ===========================
# FILE PROCESSING & DOWNLOADING
# ===========================

def process_files(drive_service, files, required_columns):
    combined_data = []
    summary_report = {"Resumed Files": [], "Fully Downloaded Files": [], "Skipped Files": []}

    try:
        with open(CHECKPOINT_FILE, "r") as f:
            download_progress = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        download_progress = {}

    for file in files:
        file_id = file['id']
        file_name = file['name']
        file_size = int(file.get('size', 0))
        bytes_downloaded = download_progress.get(file_name, 0)

        if bytes_downloaded >= file_size:
            print(f"⏩ {file_name} already downloaded. Skipping...")
            summary_report["Skipped Files"].append(file_name)
            continue

        print(f"\n⬇️ Downloading file: {file_name} (Resuming from {bytes_downloaded} bytes)")

        request = drive_service.files().get_media(fileId=file_id)
        file_stream = BytesIO()
        downloader = googleapiclient.http.MediaIoBaseDownload(file_stream, request, chunksize=CHUNKSIZE)
        
        try:
            while True:
                status, done = downloader.next_chunk()
                bytes_downloaded += len(file_stream.getvalue())
                download_progress[file_name] = bytes_downloaded

                # Save progress
                with open(CHECKPOINT_FILE, "w") as f:
                    json.dump(download_progress, f)

                print(f"✅ Download {int(status.progress() * 100)}% complete for {file_name}.")
                
                if done:
                    file_stream.seek(0)
                    df = pd.read_csv(file_stream, usecols=required_columns)
                    df['source_file'] = file_name
                    combined_data.append(df)
                    summary_report["Fully Downloaded Files"].append(file_name)
                    del download_progress[file_name]
                    with open(CHECKPOINT_FILE, "w") as f:
                        json.dump(download_progress, f)
                    break
        except (requests.ConnectionError, HttpError) as e:
            print(f"❌ Network error during download: {e}. Saving progress...")
            summary_report["Resumed Files"].append(file_name)

    if combined_data:
        final_df = pd.concat(combined_data, ignore_index=True)
        final_df[['to_qty']] = final_df[['to_qty']].fillna(0)
        return final_df, summary_report
    else:
        return pd.DataFrame(), summary_report


In [6]:
# ===========================
# MAIN EXECUTION FLOW
# ===========================

if __name__ == "__main__":
    drive_service = authenticate_drive_service()

    local_tz = pytz.timezone("Asia/Kolkata")
    start_date, end_date = get_date_range(local_tz, days= 79)
    date_range = pd.date_range(start=start_date, end=end_date).strftime("%Y%m%d").tolist()

    folder_id = '1akFe2_iKCqoZ0lAETRipXxDSKYnFmx2x'
    filtered_files = fetch_files_with_dates(drive_service, folder_id, date_range)

    required_columns = ['dd', 'mm', 'yyyy', 'mh_code', 'type', 'store_name', 'to_qty', 'store_id', 'product_variant_id']
    final_df, summary_report = process_files(drive_service, filtered_files, required_columns)

    print("\n📌 Summary Report:")
    for category, files in summary_report.items():
        print(f"{category}: {len(files)} files")


✅ Google Drive service authorized successfully.
📂 Total files found in the folder: 4255
📅 Number of files matching the date range: 692

⬇️ Downloading file: 20250330_Bengaluru.csv (26.99 MB)


20250330_Bengaluru.csv: 100%|██████████| 28.3M/28.3M [00:12<00:00, 2.26MB/s]



⬇️ Downloading file: 20250329_Bengaluru.csv (27.30 MB)


20250329_Bengaluru.csv: 100%|██████████| 28.6M/28.6M [00:11<00:00, 2.49MB/s]



⬇️ Downloading file: 20250329_NCR.csv (47.82 MB)


...


⬇️ Downloading file: 20250323_Bengaluru.csv (29.02 MB)


20250323_Bengaluru.csv: 100%|██████████| 30.4M/30.4M [00:11<00:00, 2.67MB/s]



⬇️ Downloading file: 20250322_HYD.csv (16.41 MB)


20250322_HYD.csv:  61%|██████    | 10.5M/17.2M [00:05<00:03, 1.98MB/s]


KeyboardInterrupt: 

In [12]:

from datetime import datetime

# Get today's date
today = datetime.now()

# Calculate the day of the year
day_of_year = today.timetuple().tm_yday

# Display the result
print(f"Total days till date in the current year: {day_of_year}")


Total days till date in the current year: 79
